# 05 – Lookback Options Pricing

In this notebook, we explore **lookback options**, a class of exotic derivatives whose payoff depends on the historical maximum or minimum price of the underlying asset over the option’s lifetime.

We implement pricing via **Monte Carlo simulation** using path tracking, and provide comparisons with market intuition and analytical insights.

---

### Types of Lookback Options

- **Floating Strike Lookback Call**: Payoff = max(S) - S(T)
- **Floating Strike Lookback Put**: Payoff = S(T) - min(S)
- **Fixed Strike Lookback Call**: Payoff = max(S) - K
- **Fixed Strike Lookback Put**: Payoff = K - min(S)

---

We begin with the Monte Carlo implementation for the most basic cases.


In [ ]:
import numpy as np

def monte_carlo_lookback(
    S: float,
    K: float,
    T: float,
    r: float,
    sigma: float,
    option_type: str = "call",
    strike_type: str = "fixed",
    n_simulations: int = 10000,
    n_steps: int = 100
) -> float:
    """
    Prices European-style lookback options using Monte Carlo simulation.
    """

    # --- Input validation ---
    if option_type not in {"call", "put"}:
        raise ValueError("option_type must be either 'call' or 'put'")
    if strike_type not in {"fixed", "floating"}:
        raise ValueError("strike_type must be either 'fixed' or 'floating'")
    if any(param <= 0 for param in [S, T, sigma, n_simulations, n_steps]):
        raise ValueError("S, T, sigma, n_simulations and n_steps must be positive.")

    dt = T / n_steps
    discount = np.exp(-r * T)


    # Simulate asset paths
    Z = np.random.normal(0, 1, size=(n_simulations, n_steps))
    paths = np.zeros_like(Z)
    paths[:, 0] = S

    drift = (r - 0.5 * sigma**2) * dt
    diffusion = sigma * np.sqrt(dt)


    for t in range(1, n_steps):
        paths[:, t] = paths[:, t-1] * np.exp(drift + diffusion * Z[:, t])

    S_T = paths[:, -1]
    S_max = np.max(paths, axis=1)
    S_min = np.min(paths, axis=1)


    # Compute payoffs
    if strike_type == "fixed":
        payoffs = (
            np.maximum(S_max - K, 0) if option_type == "call"
            else np.maximum(K - S_min, 0)
        )
    else:  # floating
        payoffs = (
            np.maximum(S_T - S_min, 0) if option_type == "call"
            else np.maximum(S_max - S_T, 0)
        )

    return discount * np.mean(payoffs)


#### Examples

In [ ]:
# Parameters AAPL
S = 211.16
K = 210.0             # close to the current price
T = 0.25              # 3 month ≃ 0.25 years
r = 0.05              # 5 %
sigma = 0.286         # 28.6 % (implied volatility)

n_sim = 5000
n_steps = 100


# Run different scenarios
fixed_call = monte_carlo_lookback(S, K, T, r, sigma, option_type='call', strike_type='fixed', n_simulations=n_sim, n_steps=n_steps)
fixed_put = monte_carlo_lookback(S, K, T, r, sigma, option_type='put', strike_type='fixed', n_simulations=n_sim, n_steps=n_steps)
floating_call = monte_carlo_lookback(S, K, T, r, sigma, option_type='call', strike_type='floating', n_simulations=n_sim, n_steps=n_steps)
floating_put = monte_carlo_lookback(S, K, T, r, sigma, option_type='put', strike_type='floating', n_simulations=n_sim, n_steps=n_steps)


# Show results
print("Lookback Option Prices (Monte Carlo):")
print(f"Fixed Strike Call:     {fixed_call:.4f}")
print(f"Fixed Strike Put:      {fixed_put:.4f}")
print(f"Floating Strike Call:  {floating_call:.4f}")
print(f"Floating Strike Put:   {floating_put:.4f}")


Lookback Option Prices (Monte Carlo):
Fixed Strike Call:     25.4863
Fixed Strike Put:      19.0815
Floating Strike Call:  22.6628
Floating Strike Put:   21.6295


### 3d Graphic

In [ ]:
import plotly.graph_objects as go


# Grid of parameters
sigmas = np.linspace(0.05, 0.5, 10)
maturities = np.linspace(0.1, 2.0, 10)
prices_grid = np.zeros((len(sigmas), len(maturities)))


# Compute option prices for each (sigma, T) pair
for i, sigma_i in enumerate(sigmas):
    for j, T_j in enumerate(maturities):
        price = monte_carlo_lookback(
            S=S,
            K=K,
            T=T_j,
            r=r,
            sigma=sigma_i,
            option_type="call",
            strike_type="floating",
            n_simulations=5000,  # reduce for faster preview
            n_steps=100
        )
        prices_grid[i, j] = price


# Create meshgrid for plotting
Sigma, Tgrid = np.meshgrid(maturities, sigmas)


# Plot
fig = go.Figure(data=[go.Surface(
    z=prices_grid,
    x=Tgrid,
    y=Sigma,
    colorscale='Viridis'
)])


fig.update_layout(
    title="Lookback Floating Call Price vs Maturity & Volatility",
    scene=dict(
        xaxis_title='Maturity (T)',
        yaxis_title='Volatility (σ)',
        zaxis_title='Option Price'
    ),
    height=600,
    width=800
)


fig.show()
